## Setup File for Keras Models
Use `%run Setup.ipynb` in another notebook to perform all these tasks automatically.

Parameters that can be re-configured:

In [1]:
MAX_NB_WORDS = 40000 # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 30 # max length of text (words) including padding
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 200 # embedding dimensions for word vectors (word2vec/GloVe)
GLOVE_DIR = dir+"dataset/glove/glove.twitter.27B."+str(200)+"d.txt"
print("[i] Loaded Parameters:\n",
      MAX_NB_WORDS,MAX_SEQUENCE_LENGTH+5,
      VALIDATION_SPLIT,EMBEDDING_DIM,"\n",
      GLOVE_DIR)

TypeError: ignored

Imports:

In [ ]:
print("[i] Importing Modules...")
import numpy as np
import pandas as pd
import re, sys, os, csv, keras, pickle

[i] Importing Modules...


In [ ]:
from keras import regularizers, initializers, optimizers, callbacks
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.layers import Layer, InputSpec
print("[+] Using Keras version",keras.__version__)

[+] Using Keras version 2.12.0


In [ ]:
print("[+] Finished Importing Modules")

[+] Finished Importing Modules


In [ ]:
texts, labels = [], []
print("[i] Reading from csv file...", end="")
with open(dir+'data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        texts.append(row[0])
        labels.append(row[1])
print("Done!")

[i] Reading from csv file...Done!


Convert text to word tokens (numbers that refer to the words)

In [ ]:
"""
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("[i] Saved word tokenizer to file: tokenizer.pickle")
"""

'\ntokenizer = Tokenizer(num_words=MAX_NB_WORDS)\ntokenizer.fit_on_texts(texts)\nwith open(\'tokenizer.pickle\', \'wb\') as handle:\n    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)\nprint("[i] Saved word tokenizer to file: tokenizer.pickle")\n'

In [ ]:
with open(dir+'tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

Convert tweets to sequences of word tokens with zero padding at the front and back

In [ ]:
sequences = tokenizer.texts_to_sequences_generator(list(map(lambda x:x.split(),texts)))
word_index = tokenizer.word_index
print('[i] Found %s unique tokens.' % len(word_index))
data_int = pad_sequences(list(sequences), padding='pre', maxlen=(MAX_SEQUENCE_LENGTH-5))
data = pad_sequences(data_int, padding='post', maxlen=(MAX_SEQUENCE_LENGTH))

[i] Found 34359 unique tokens.


array([   0,    0,    0,    0,    0,   44,   27,   42,  115,   98, 1637,
         98, 3230,   98, 8802,   98, 1070,   98,  194,  219,  178,    5,
         31,   32,  445,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
labels = to_categorical(np.asarray(labels)) # convert to one-hot encoding vectors
print('[+] Shape of data tensor:', data.shape)
print('[+] Shape of label tensor:', labels.shape)

[+] Shape of data tensor: (1, 30)
[+] Shape of label tensor: (47288, 5, 2)


In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [ ]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('[i] Number of entries in each category:')
print("[+] Training:\n",y_train.sum(axis=0))
print("[+] Validation:\n",y_val.sum(axis=0))

[i] Number of entries in each category:
[+] Training:
 [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[+] Validation:
 [[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


### Preparing the Embedding layer

Compute an index mapping words to known embeddings, by parsing the data dump of pre-trained embeddings.

We use pre-trained [GloVe](https://nlp.stanford.edu/projects/glove/) vectors from Stanford NLP. For new words, a "randomised vector" will be created.

In [ ]:
x_train[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
embeddings_index = {}
f = open(GLOVE_DIR)
print("[i] Loading GloVe from:",GLOVE_DIR,"...",end="")
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
print("Done.\n[+] Proceeding with Embedding Matrix...", end="")
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("[i] Completed!")

In [ ]:
print("[i] Finished running setup.")